# Algoritmos de optimización - Seminario<br>
<b>Nombre y Apellidos: </b> Yeray Quiles Ferrández <br>
> 1. Organizar sesiones de doblaje <br>


## Enunciado
<b>Descripción del problema: </b><br>
Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible. Los datos son:
- <b>Número de actores: </b> `10`
- <b> Número de tomas: </b> `30`
- <b>Actores/Tomas:</b> https://docs.google.com/spreadsheets/d/1Ipn6IrbQP4ax8zOnivdBIw2lN0JISkJG4fXndYd27U0/edit?gid=0#gid=0
         

#### Obtenemos los datos de la tabla

In [34]:
import csv
import urllib.request

# Descargar datos de Google Sheets
sheet_id = "1Ipn6IrbQP4ax8zOnivdBIw2lN0JISkJG4fXndYd27U0"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv"

# Leer y procesar
tomas_actores = {}
with urllib.request.urlopen(url) as response:
    content = response.read().decode('utf-8')
    lines = content.strip().split('\n')
    reader = csv.reader(lines)
    
    # Procesar cada línea
    for row in reader:
        if row and row[0].isdigit():  # Solo procesar si el primer elemento es un número
            toma = int(row[0])
            # Los actores están en las columnas 1-10
            actores = {i for i in range(1, 11) if row[i] == '1'}
            tomas_actores[toma] = actores

En la tabla: 
- `1` el actor participa en la toma
- `0` el actor NO participa en la toma     


| Toma | Actor 1 | Actor 2 | Actor 3 | Actor 4 | Actor 5 | Actor 6 | Actor 7 | Actor 8 | Actor 9 | Actor 10 | Total |
|------|---------|---------|---------|---------|---------|---------|---------|---------|---------|----------|-------|
| 1 | 1 | 1 | 1 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 5 |
| 2 | 0 | 0 | 1 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 3 |
| 3 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 3 |
| 4 | 1 | 1 | 0 | 0 | 0 | 0 | 1 | 1 | 0 | 0 | 4 |
| 5 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 3 |
| 6 | 1 | 1 | 0 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 4 |
| 7 | 1 | 1 | 0 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 4 |
| 8 | 1 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 3 |
| 9 | 1 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 3 |
| 10 | 1 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 4 |
| 11 | 1 | 1 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 5 |
| 12 | 1 | 1 | 1 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 5 |
| 13 | 1 | 0 | 0 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 3 |
| 14 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 3 |
| 15 | 1 | 1 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 3 |
| 16 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 1 | 2 |
| 17 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 2 |
| 18 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 2 |
| 19 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 2 |
| 20 | 1 | 0 | 1 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 4 |
| 21 | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 2 |
| 22 | 1 | 1 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 4 |
| 23 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 2 |
| 24 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 2 |
| 25 | 1 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 1 | 4 |
| 26 | 1 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 4 |
| 27 | 0 | 0 | 0 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 2 |
| 28 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 2 |
| 29 | 1 | 0 | 0 | 0 | 1 | 1 | 0 | 0 | 0 | 0 | 3 |
| 30 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 2 |
|------|---------|---------|---------|---------|---------|---------|---------|---------|---------|----------|-------|
| **TOTAL** | **22** | **14** | **13** | **15** | **11** | **8** | **3** | **4** | **2** | **2** | **94** |


## ¿Cuántas posibilidades hay sin tener en cuenta las restricciones? 

El objetivo sería organizar el doblaje de una película con un total de 30 tomas y 10 actores. Para calcular el número de posibilidades sin restricciones, es necesario estudiar el orden de las tomas a grabar y la distribución de las sesiones de grabación a lo largo de los días.
---

### Opción 1: n! (Permutaciones)

Si únicamente consideramos el **orden de las tomas**:
- Tendríamos n! = 30! permutaciones posibles.
- Este enfoque asume que el orden de grabación es lo único relevante para la planificación.
- No considera cómo se distribuyen las tomas en múltiples días de grabación.
- Sería correcto solo si todas las tomas se graban en un único día.

**Resultado:** 265,252,859,812,191,058,636,308,480,000,000 posibilidades

Esta opción no es suficiente para modelar el problema real de coordinación de doblaje, donde la distribución temporal en días es un factor esencial de la planificación.



In [3]:
import math

# Parámetros del problema
n = 30

# Opción 1: Solo n! (descartada)
solo_factorial = math.factorial(n)

print(f'Opción 1: n! = {solo_factorial:,}')


Opción 1: n! = 265,252,859,812,191,058,636,308,480,000,000



### Opción 2: n! × 2^(n-1) (Secuenciación + Distribución)

#### Orden de las tomas
Si asumimos que el número total de ordenaciones posibles es relevante, entonces se trata de un problema de **permutaciones sin repetición** y el número de posibilidades es **n!**.

#### Distribución de las sesiones de grabación
- En cada sesión de grabación de un día, se pueden grabar un número distinto de tomas.
- Las grabaciones son únicas, es decir, no pueden repetirse. Por tanto, una vez grabada una toma se eliminará del conjunto de tomas a grabar en las siguientes sesiones.
- Sea d∈{0,…,n−1} el número de particiones en que se pueden distribuir las sesiones de grabación de las tomas. Por ejemplo:
  - Si se graban todas las tomas en un solo día: **d=0** (sin particiones)
  - Si las tomas se graban en dos sesiones: **d=1** (una partición)
  - Y así sucesivamente
- El número de posibles ordenaciones de la partición d=0 sería C(n-1,0), el de la partición d=1 sería C(n-1,1), y así hasta el último número combinatorio.
- Por tanto, el número total de posibles particiones será la suma de todas las posibles distribuciones: **∑(j=0 hasta n-1) C(n-1,j)**

#### Número total de posibilidades sin restricción

El número total de posibles ordenaciones de las tomas a lo largo de los días se obtiene calculando el **producto** del orden de las tomas por la distribución de las sesiones de grabación, que por **identidad binomial** es:

**Núm. posibilidades = n! × ∑(j=0 hasta n-1) C(n-1,j) = n! × 2^(n-1)**


In [ ]:
import math 

# Parámetros del problema
n = 30

# Opción 2: n! × 2^(n-1) 
orden_tomas = math.factorial(n)
distribucion_dias = 2**(n-1)
total_posibilidades = orden_tomas * distribucion_dias

# Resultados
print(f'Opción 2: n! × 2^(n-1) = {total_posibilidades:,}')
print(f'Factor de diferencia: {total_posibilidades/solo_factorial:,}')

Opción 2: n! × 2^(n-1) = 142,406,544,757,979,162,368,320,409,970,933,760,000,000
Factor de diferencia: 536,870,912.0



### Comparación de Enfoques

| Aspecto | n! (Opción 1) | n! × 2^(n-1) (Opción 2) |
|---------|---------------|-------------------------|
| **Considera orden de tomas** | ✅ Sí | ✅ Sí |
| **Considera distribución en días** | ❌ No | ✅ Sí |
| **Realismo para doblaje** | ❌ Limitado | ✅ Completo |
| **Resultado para n=30** | ~2.65 × 10^32 | ~1.42 × 10^41 |

### Resultados

| Opción | Fórmula | Resultado |
|--------|---------|-----------|
| **Opción 1 (descartada)** | n! | 265,252,859,812,191,058,636,308,480,000,000 |
| **Opción 2 (correcta)** | n! × 2^(n-1) | 142,406,544,757,979,162,368,320,409,970,933,760,000,000 |

La opción correcta considera tanto el orden de grabación como la distribución temporal, siendo 536,870,912 veces mayor que la opción que solo considera el orden.

**Respuesta final:** 142,406,544,757,979,162,368,320,409,970,933,760,000,000 posibilidades

## ¿Cuántas posibilidades hay teniendo en cuenta todas las restricciones?

Se busca coordinar el doblaje de una película con 30 tomas y 10 actores, con un máximo de 6 tomas por día y el objetivo de minimizar el coste total. Para calcular el número de posibilidades con restricciones, es necesario estudiar el orden de las tomas a grabar y la distribución de las sesiones de grabación a lo largo de los días.

---

### Opción 1: Combinaciones con orden de días relevante

Si consideramos que el orden cronológico de los días es importante:

#### Orden de las tomas
- Dado que no tenemos restricciones sobre cómo deben ser grabadas las tomas dentro de la sesión de un día, se asume que el orden de grabación de las k tomas dentro de cada día es indiferente.
- Sin embargo, el orden de los días sí se considera relevante (Día 1 ≠ Día 2 ≠ Día 3...).

#### Distribución de las sesiones de grabación
- El mínimo número de sesiones necesarias es **r = ⌈n/k⌉ = ⌈30/6⌉ = 5** días.
- Para optimizar costes, asumimos sesiones de **k = 6** tomas cada una.
- Las grabaciones son únicas: una vez grabada una toma debe ser excluida del conjunto de tomas disponibles.
- Para el día **j+1**, tenemos **C(n-k·j, k)** posibilidades, donde **j ∈ {0,1,2,3,4}**.

#### Número total de posibilidades (orden relevante)

**Núm. posibilidades = ∏[j=0 hasta 4] C(30-6·j, 6)**

**Resultado:** 1,370,874,167,589,326,400 posibilidades

Esta opción distingue entre configuraciones idénticas en términos de coste pero diferentes en orden temporal.

### Opción 2: Combinaciones con orden de días irrelevante

Esta es la opción escogida para el problema de doblaje, considerando:

#### Orden de las tomas
- El orden de grabación de las k tomas dentro de cada día es indiferente.
- El orden de los días tampoco es relevante para el objetivo de minimización de costes.
- Grabar el conjunto {1,2,3,4,5,6} el lunes vs. el martes no afecta el coste total.

#### Distribución de las sesiones de grabación
- Mantenemos la configuración de 5 días con 6 tomas cada uno.
- Las configuraciones que solo difieren en el orden de los días se consideran equivalentes.
- Por ejemplo: [Día A: {1,2,3,4,5,6}, Día B: {7,8,9,10,11,12}] ≡ [Día A: {7,8,9,10,11,12}, Día B: {1,2,3,4,5,6}]

#### Corrección por equivalencia de días

Dado que el orden de los días no importa, debemos dividir el resultado de la Opción 1 por el número de permutaciones de los 5 días:

**Núm. posibilidades = [∏[j=0 hasta 4] C(30-6·j, 6)] ÷ 5!**

### Comparación de Enfoques

| Aspecto | Opción 1 (Orden relevante) | Opción 2 (Orden irrelevante) |
|---------|----------------------------|-------------------------------|
| **Considera restricción k≤6** | ✅ Sí | ✅ Sí |
| **Considera objetivo de costes** | ❌ Parcialmente | ✅ Completamente |
| **Distingue días equivalentes** | ❌ Sí  | ✅ No  |
| **Realismo para doblaje** | ❌ Limitado | ✅ Completo |
| **Resultado para n=30, k=6** | ~1.37 × 10¹⁵ | ~1.14 × 10¹³ |


### Resultados

| Opción | Fórmula | Resultado |
|--------|---------|-----------|
| **Opción 1** | ∏[j=0 hasta 4] C(30-6·j, 6) | 1,370,874,167,589,326,400 |
| **Opción 2** | [∏[j=0 hasta 4] C(30-6·j, 6)] ÷ 5! | 11,423,951,396,577,720 |

La opción correcta evita el sobreconteo de configuraciones equivalentes, siendo 120 veces menor que la opción que distingue incorrectamente el orden de días.


**Respuesta final:** 11,423,951,396,577,720 posibilidades

In [1]:
import math

n = 30
k = 6
num_dias = n // k  # = 5 días

# Cálculo del producto de combinaciones
num_posibilidades_con_orden = 1
for j in range(0, num_dias):
    combinacion = math.comb(n - k*j, k)
    num_posibilidades_con_orden *= combinacion
    print(f"C({n - k*j}, {k}) = {combinacion:,}")

# Corrección por orden irrelevante de días
factorial_dias = math.factorial(num_dias)
num_posibilidades_sin_orden = num_posibilidades_con_orden // factorial_dias

print(f"Con orden: {num_posibilidades_con_orden:,}")
print(f"Factorial 5!: {factorial_dias}")
print(f"Sin orden: {num_posibilidades_sin_orden:,}")


C(30, 6) = 593,775
C(24, 6) = 134,596
C(18, 6) = 18,564
C(12, 6) = 924
C(6, 6) = 1
Con orden: 1,370,874,167,589,326,400
Factorial 5!: 120
Sin orden: 11,423,951,396,577,720


## ¿Cual es la estructura de datos que mejor se adapta al problema?

### Primera aproximación para definir la estructura de los datos:

**Datos de entrada**

Inicialmente se pensó en trabajar con la matriz de datos como un **array bidimensional de numpy** o un **dataframe de pandas** de manera que:
* en las filas tuviese como índice el número de toma (1 a 30),
* en las columnas tuviese como nombre el número de actor (1 a 10) y  
* en cada valor de la matriz el indicador ∈ {1 si en la toma i-ésima participa el actor j-ésimo, 0 en caso contrario}


**Datos de salida**

La solución se presentaría como una lista de sublistas donde cada sublista estaría formada por un número de tomas (≤6 tomas) para una sesión específica y se presentaría tantas sublistas como sesiones de grabación se considerase. Por ejemplo:

`[[1, 2, 3, 4, 5, 6], [7, 8, 9, 10, 11, 12], [13, 14, 15, 16, 17, 18], [19, 20, 21, 22, 23, 24], [25, 26, 27, 28, 29, 30]]`

No obstante, al experimentar las primeras dificultades en los diferentes bucles de optimización y analizar el rendimiento de la primera aproximación propuesta, se tomó la decisión de considerar una estructura alternativa.

### Problemas identificados con la aproximación inicial:

Durante la implementación inicial surgieron varios problemas:

1. **Ineficiencia en consultas frecuentes**: La operación más común "¿qué actores necesito para las tomas {1,5,12,18,23,29}?" requería iterar por cada toma y cada actor (6×10=60 operaciones), cuando conceptualmente solo necesitaba unir 6 conjuntos pequeños.

2. **Desperdicio masivo de memoria**: Del total de 300 posiciones en la matriz (30×10), solo 94 contenían el valor 1, significando que el 68.7% del espacio almacenaba ceros innecesarios.

3. **Complejidad en operaciones de conjuntos**: Calcular la unión de actores para múltiples tomas requería bucles anidados y creación manual de conjuntos intermedios, complicando el código y ralentizando las evaluaciones.

4. **Escalabilidad limitada**: Para problemas mayores, el desperdicio de memoria crecería proporcionalmente, haciendo inviable la estructura para instancias más grandes.

### Estructura final de los datos:

**Datos de entrada**

En base a la matriz descrita en el apartado anterior se llevó a cabo una transformación para montar una **estructura híbrida de diccionarios con conjuntos** donde:
* cada clave es un número de toma (1 a 30) y
* cada valor del diccionario es un **conjunto (set)** con los números de actores que participan en esa toma.

Por ejemplo:
```python
{
    1: {1, 2, 3, 4, 5},     # Toma 1: actores 1,2,3,4,5
    2: {3, 4, 5},           # Toma 2: actores 3,4,5  
    3: {2, 5, 7},           # Toma 3: actores 2,5,7
    # ... hasta toma 30
}
```

Esta estructura elimina completamente el almacenamiento de ceros y aprovecha las operaciones nativas de conjuntos de Python para uniones eficientes.


### Ventajas de la estructura final:

1. **Memoria optimizada**: Solo almacena las 94 participaciones reales vs 300 posiciones de la matriz.

2. **Operaciones más rápidas**: La consulta crítica se reduce de O(k×m) a O(k) operaciones.

3. **Código más limpio**: Las operaciones de conjuntos son naturales en Python y eliminan bucles complejos.

4. **Escalabilidad natural**: El crecimiento es proporcional a las participaciones reales, no al producto cartesiano teórico.

### Limitaciones

Debido a la magnitud de los cálculos requeridos para evaluar **11,423,951,396,577,720 configuraciones posibles** y las restricciones computacionales, se introdujeron varias consideraciones:

1. **Se asumió que el orden de los días no es relevante** para el objetivo de minimización de costes, dividiendo el espacio de soluciones por 5! para evitar sobreconteo de configuraciones equivalentes.

2. **Se estableció uniformidad en el número de tomas por sesión** (k=6) para la configuración óptima, aunque la estructura permite flexibilidad para otras distribuciones.

3. **Se priorizó simplicidad sobre optimización extrema**, manteniendo el código legible y mantenible en lugar de implementar estructuras más complejas pero marginalmente más rápidas.


La estructura final representa un equilibrio óptimo entre eficiencia computacional, simplicidad implementativa y escalabilidad para el problema específico de coordinación de doblaje planteado.

In [6]:
tomas_actores = {
    1: {1, 2, 3, 4, 5},     2: {3, 4, 5},           3: {2, 5, 7},
    4: {1, 2, 7, 8},        5: {2, 4, 8},           6: {1, 2, 4, 5},
    7: {1, 2, 4, 5},        8: {1, 2, 6},           9: {1, 2, 4},
    10: {1, 2, 6, 9},       11: {1, 2, 3, 5, 8},    12: {1, 2, 3, 4, 6},
    13: {1, 4, 5},          14: {1, 3, 6},          15: {1, 2, 7},
    16: {4, 10},            17: {1, 3},             18: {3, 6},
    19: {1, 3},             20: {1, 3, 4, 5},       21: {6, 8},
    22: {1, 2, 3, 4},       23: {1, 3},             24: {3, 6},
    25: {1, 2, 4, 10},      26: {1, 3, 5, 9},       27: {4, 5},
    28: {1, 4},             29: {1, 5, 6},          30: {1, 4}
}

## ¿Cuál es la función objetivo?

La función objetivo representa el **corazón del problema de optimización** de coordinación de doblaje. Su diseño debe capturar fielmente la realidad económica del proceso: minimizar el coste total de producción expresado en **días-actor**, donde cada unidad representa el coste de un actor trabajando durante una sesión completa de grabación.

---

## Contexto
En el proceso de doblaje, los actores cobran una **tarifa fija por día** independientemente de cuántas tomas graben en esa jornada. Esto significa que si un actor participa en una sola toma del día, debe cobrar el día completo igual que si participara en las seis tomas máximas permitidas. Esta característica fundamental del dominio determina que la función objetivo debe enfocarse en **minimizar el número total de actores únicos** que trabajan cada día, no el número de participaciones individuales.

El desafío principal radica en encontrar agrupaciones de tomas que **maximicen la reutilización de actores** entre las tomas de una misma sesión, reduciendo así el número de actores diferentes que deben ser convocados cada día.

La función objetivo que se implementó, llamada `coste`, determina el coste total asociado a una solución propuesta para la programación de las sesiones de doblaje. La función calcula cuántos actores únicos participan en cada sesión y suma los costes de todas las sesiones para obtener el coste total de la configuración.

### Cálculo de la función

La función `coste` implementa un algoritmo sencillo pero eficaz que refleja la realidad del problema:

**1. Iteración por sesiones**: Para cada sesión de grabación en la configuración, se calcula independientemente su coste asociado.

**2. Unión de actores por sesión**: Dentro de cada sesión, se identifican todos los actores únicos necesarios mediante la unión de los conjuntos de actores de cada toma. Esta operación aprovecha la eficiencia de la estructura de datos de diccionarios+conjuntos desarrollada anteriormente.

**3. Cálculo de coste marginal**: El coste de cada sesión se obtiene multiplicando el número de actores únicos por el salario diario. En el caso base, se asume salario_diario=1 para obtener directamente el número de días-actor.

**4. Agregación final**: El coste total se calcula sumando los costes marginales de todas las sesiones.



### Interpretación

La función `resumen_configuracion` proporciona un análisis legible de cualquier configuración propuesta, mostrando:

- **Desglose por sesión**: Qué tomas se graban juntas y qué actores son necesarios
- **Coste marginal**: Cuánto cuesta cada sesión individual
- **Coste total**: El valor objetivo que se busca minimizar


In [ ]:

def coste(configuracion, tomas_actores, salario_diario=1):
    """
    Calcula el coste total de una configuración de sesiones de doblaje.
    """
    costes_por_sesion = []
    
    for sesion in configuracion:
        actores_sesion = set()
        for toma in sesion:
            actores_sesion.update(tomas_actores[toma])
        costes_por_sesion.append(len(actores_sesion) * salario_diario)
    
    coste_total = sum(costes_por_sesion)
    return (coste_total, costes_por_sesion)

def resumen_configuracion(configuracion, tomas_actores):
    """
    Genera un resumen detallado de la configuración.
    """
    coste_total, costes_por_sesion = coste(configuracion, tomas_actores)
    
    print("RESUMEN DE CONFIGURACIÓN")
    print("="*30)
    
    for i, (sesion, coste_dia) in enumerate(zip(configuracion, costes_por_sesion), 1):
        actores_sesion = set()
        for toma in sesion:
            actores_sesion.update(tomas_actores[toma])
        
        print(f"Sesión {i}: tomas {sesion}")
        print(f"         actores {sorted(actores_sesion)} → {coste_dia} días-actor")
    
    print("="*30)
    print(f"TOTAL: {coste_total} días-actor")
    
    return coste_total

# EJEMPLO
config_secuencial = [
    [1, 2, 3, 4, 5, 6],       # Día 1
    [7, 8, 9, 10, 11, 12],    # Día 2  
    [13, 14, 15, 16, 17, 18], # Día 3
    [19, 20, 21, 22, 23, 24], # Día 4
    [25, 26, 27, 28, 29, 30]  # Día 5
]

coste_seq = resumen_configuracion(config_secuencial, tomas_actores)
print(f"Coste secuencial: {coste_seq} días-actor\n")

RESUMEN DE CONFIGURACIÓN
Sesión 1: tomas [1, 2, 3, 4, 5, 6]
         actores [1, 2, 3, 4, 5, 7, 8] → 7 días-actor
Sesión 2: tomas [7, 8, 9, 10, 11, 12]
         actores [1, 2, 3, 4, 5, 6, 8, 9] → 8 días-actor
Sesión 3: tomas [13, 14, 15, 16, 17, 18]
         actores [1, 2, 3, 4, 5, 6, 7, 10] → 8 días-actor
Sesión 4: tomas [19, 20, 21, 22, 23, 24]
         actores [1, 2, 3, 4, 5, 6, 8] → 7 días-actor
Sesión 5: tomas [25, 26, 27, 28, 29, 30]
         actores [1, 2, 3, 4, 5, 6, 9, 10] → 8 días-actor
TOTAL: 38 días-actor
Coste secuencial: 38 días-actor



### Relación con el problema de optimización global

Esta función objetivo proporciona la métrica de calidad necesaria para evaluar cualquier configuración propuesta y será fundamental para implementar algoritmos que comparen las 11,423,951,396,577,720 configuraciones posibles buscando minimizar el coste total.

## ¿Es un problema de maximización o minimización?

Este caso es un **problema de minimización**. El objetivo consiste en encontrar la configuración de sesiones de doblaje que **minimice el coste total** expresado en días-actor.
El enunciado pide que se busque "*planificar las sesiones por día de manera que **el gasto por los servicios de los actores de doblaje sea el menor posible***", por tanto estamos ante minimización de costes.


### Diferenciación con problemas de maximización

Por el contrario, si fuera un problema de maximización, estariamos hablando de:
- **Maximizar la utilización** de actores por sesión
- **Maximizar el número de tomas** por sesión
- **Maximizar la eficiencia** temporal del proceso


### Verificación con el resultado obtenido

La configuración secuencial de referencia produce **38 días-actor**. El objetivo del  algoritmo de optimización será encontrar configuraciones alternativas que produzcan un coste **< 38 días-actor**, confirmando que es un problema de minimización.



## Diseña un algoritmo para resolver el problema por fuerza bruta

El algoritmo de fuerza bruta consiste en **evaluar todas las configuraciones posibles** y seleccionar la que tenga menor coste.

### Algoritmo conceptual

```
1. Generar todas las formas de agrupar 30 tomas en sesiones de ≤6 tomas
2. Para cada configuración: calcular su coste usando la función coste()  
3. Retornar la configuración con menor coste
```

### Implementación simplificada

Debido a las **11,423,951,396,577,720 configuraciones posibles**, implementamos una versión que evalúa una muestra aleatoria:


In [ ]:
import random

def fuerza_bruta_simple(tomas_actores, num_evaluaciones=1000):
    """
    Algoritmo de fuerza bruta. Evalúa una muestra de configuraciones aleatorias.
    """
    mejor_configuracion = None
    mejor_coste = float('inf')
    
    for _ in range(num_evaluaciones):
        # Generar configuración aleatoria
        configuracion = generar_configuracion_aleatoria()
        
        # Evaluar coste
        coste_actual, _ = coste(configuracion, tomas_actores)
        
        # Si es mejor, guardarla
        if coste_actual < mejor_coste:
            mejor_coste = coste_actual
            mejor_configuracion = configuracion
    
    return mejor_configuracion, mejor_coste

def generar_configuracion_aleatoria():
    """Genera una configuración aleatoria válida"""
    tomas = list(range(1, 31))  # Tomas 1-30
    random.shuffle(tomas)
    
    configuracion = []
    i = 0
    while i < len(tomas):
        # Tomar entre 1-6 tomas para esta sesión
        num_tomas = min(6, len(tomas) - i)
        sesion = tomas[i:i + num_tomas]
        configuracion.append(sesion)
        i += num_tomas
    
    return configuracion

# Ejemplo de uso
print("🔍 Ejecutando fuerza bruta simplificado...")
mejor_config, mejor_coste = fuerza_bruta_simple(tomas_actores, 1000)

print(f"✅ Mejor coste encontrado: {mejor_coste} días-actor")
print(f"📋 Configuración: {mejor_config}")


🔍 Ejecutando fuerza bruta simplificado...
✅ Mejor coste encontrado: 33 días-actor
📋 Configuración: [[1, 17, 22, 12, 14, 16], [28, 13, 7, 9, 27, 23], [5, 19, 6, 2, 11, 20], [10, 21, 4, 30, 25, 8], [15, 24, 18, 29, 26, 3]]


#### Algoritmo Teórico  (No ejecutable)

In [ ]:
def fuerza_bruta_completo(tomas_actores):
    """
    Evaluaría las 11,423,951,396,577,720
    """
    mejor_coste = float('inf')
    mejor_configuracion = None
    
    # Generar TODAS las configuraciones posibles
    for configuracion in todas_las_configuraciones_posibles():
        coste_actual, _ = coste(configuracion, tomas_actores)
        
        if coste_actual < mejor_coste:
            mejor_coste = coste_actual
            mejor_configuracion = configuracion
    
    return mejor_configuracion, mejor_coste


**Conclusión**: El algoritmo es conceptualmente simple pero computacionalmente inviable, justificando la necesidad de algoritmos más eficientes.

## Calcula la complejidad del algoritmo por fuerza bruta

### Análisis del algoritmo teórico

**Componentes del algoritmo:**

1. **Generación de configuraciones**: Debe generar todas las formas posibles de particionar 30 tomas en sesiones de máximo 6 tomas
2. **Evaluación por configuración**: Calcular el coste mediante la función `coste()`

**Número total de configuraciones:**
- Como calculamos anteriormente: **11,423,951,396,577,720** configuraciones

**Coste por evaluación:**
- La función `coste()` itera por cada sesión y cada toma de la sesión
- Máximo 5 sesiones × máximo 6 tomas = 30 operaciones por configuración

**Complejidad total:**
```
T(n) = Número de configuraciones × Coste por evaluación
T(n) = 11,423,951,396,577,720 × O(1)
T(n) = O(11.4 × 10^13)
```

### Complejidad espacial

- **Almacenamiento de la mejor solución**: O(30) para las tomas de la mejor configuración
- **Datos de entrada**: O(30 × 10) para la matriz de participaciones  
- **Variables auxiliares**: O(1)
- **Total**: **O(30)** = **O(1)** - constante respecto al tamaño del problema

### Tiempo de ejecución estimado

In [44]:
configuraciones_totales = 11423951396577720
evaluaciones_por_segundo = 1000000  # 1 microsegundo por evaluación

tiempo_segundos = configuraciones_totales / evaluaciones_por_segundo
tiempo_anos = tiempo_segundos // (365 * 24 * 3600)

print(tiempo_anos, "años")

362.0 años


El algoritmo de fuerza bruta tiene:

- **Complejidad temporal**: **O(11.4 × 10¹³)** 
- **Complejidad espacial**: **O(1)** 
- **Tiempo de ejecución**: **≈ 362 años**

Esta complejidad justifica la necesidad de desarrollar **algoritmos heurísticos** que proporcionen soluciones de buena calidad en tiempo razonable, sacrificando la garantía de optimalidad por la viabilidad computacional.

## Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

## Calcula la complejidad del algoritmo

## Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

## Aplica el algoritmo al juego de datos generado

## Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

## Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño